# Nullable Fields

A nullable field is slightly different from an optional field.

An optional field simply means that the data being deserialized does not need to contain the **key** for that specific field, and in that case, a pre-defined default is used.

On the other hand, nullability of a field has nothing to do with whether it is optional or not - it basically just indicates whether a field can be set to `None` (or `null` in JSON) perspective.

Let's take a look:

Since Pydantic does type validation, if we define a field to be of some type, say `int`, trying to set that field to `None` will fail, since `None` is a distinct type in Python, and cannot be coerced to an int.

In [1]:
from pydantic import BaseModel, ValidationError

class Model(BaseModel):
    field: int

In [2]:
try:
    Model(field=None)
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/int_type


As you can see, the exception we get here is saying that the data is not a valid integer, quite different from this:

In [3]:
try:
    Model()
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


where the exception has to do with a required field.

To indicate to Pydantic that a field is nullable (will entertain either the actual type, or the None type), we need to specify it in the type hint.

In [4]:
class Model(BaseModel):
    field: int | None

This type hint informs Pydantic that either an integer (or something that can be coerced to an integer) or `None` are acceptabler values for that field.

In [5]:
Model(field=None)

Model(field=None)

In [6]:
Model(field="1")

Model(field=1)

As you can see, we now have a nullable field.

But the field is not optional!

In [7]:
try:
    Model()
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


Nullable fields and optional fields often go hand in hand, simply because we often choose to set the default for a field to `None`, to indicate no value has been provided in the data.

In a case like this, we would the field to be **both** nullable and optional.

We do it this way:

In [8]:
class Model(BaseModel):
    field: int | None = None

Now we have a field that is optional (because a default has ben provided), and is nullable (because of the `int | None` type hint).

We just happened to choose `None` as a default value, but of course it could be something any integer value too.

I want to point out that the notation `int | None` only became available in Python 3.10.

Before that, we had (and still have) two additional ways of specifying this type hint.

The first one is by using `Union` from the `typing` module:

In [9]:
from typing import Union

class Model(BaseModel):
    field: Union[int, None]

This is completely equivalent to writing `int | None`.

In [10]:
Model(field=None)

Model(field=None)

Yet another way is to use `Optional`, also from the `typing` module:

In [11]:
from typing import Optional

class Model(BaseModel):
    field: Optional[int]

I typically do not use `Optional` when working with Python versions prior to 3.10.

The reason I do not like it, is that when working with Pydantic models, optional means something else. Here, we use `Optional` to indicate that field is **nullable**, not to indicate that it is optional.

We can verify this:

In [12]:
try:
    Model()
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In order to make a field optional we have to provide a default value, like this:

In [13]:
class Model(BaseModel):
    field: Optional[int] = None

In [14]:
Model()

Model(field=None)

Those tree ways of specifying nullability are completely equivalent:

In [15]:
class Model(BaseModel):
    field_1: int | None
    field_2: Union[int, None]
    field_3: Optional[int]

And, in fact, when we look at the field definitions:

In [16]:
Model.model_fields

{'field_1': FieldInfo(annotation=Union[int, NoneType], required=True),
 'field_2': FieldInfo(annotation=Union[int, NoneType], required=True),
 'field_3': FieldInfo(annotation=Union[int, NoneType], required=True)}

You'll notice that all three fields are represented by Pydantic as `Union[int, None]`

Both `int | None` and `Optional[int]` are just **convenience** syntax to represent `Union[int, None]`.

A very common mistake beginners make, is to set a field default to `None` without indicating that the field is nullable.

In other words I often see code like this:

In [17]:
class Model(BaseModel):
    field: int = None

Because of the way Pydantic does not validate defaults (by default), the above model will appear to work fine, but could cause trouble down the road:

In [18]:
m = Model()
m

Model(field=None)

But of course the field is not really nullable, so this would not work:

In [19]:
try:
    Model(field=None)
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/int_type


The correct way to define this model would be:

In [20]:
class Model(BaseModel):
    field: int | None = None

In fact, let's spend a little more time on the different combinations of optional and nullable for model fields - next video!